# Bật GPU
"Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

In [2]:
!nvidia-smi

Mon Jul 27 22:04:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cài đặt các gói cần thiết

In [3]:
!pip3 install fairseq
!pip3 install transformers

     |████████████████████████████████| 307kB 4.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 17.1MB 202kB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2021632 sha256=de01ea977981ea3d98d870b580cdc33d77a12bbca96ec16f27ed9b3dc72daab7
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
     |████████████████████████████████| 778kB 4.4MB/s 
     |████████████████████████████████| 890kB 19.2MB/s 
     |████████████████████████████████| 1.1MB 30.4MB/s 
     |████████████████████████████████| 3.0MB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e69fdf5ae23aab7909b453807a3bb8685dc49da0c6d4a6ef5777d1b787eb2740
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
root_dir = "/content/drive/My\ Drive/MasterThesis/CoLab/code_final"
src_dir = "%s/src" % root_dir
!ls $root_dir
!ls $src_dir
import sys
sys.path.append(src_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
cache	data	      src	 TRAIN1.sh    TRAIN2.ipynb
corpus	model_stage1  TRAIN0.sh  TRAIN2.bash  TRAIN3.bash
bert_tokenize.py	 __init__.py	save_vocab_file.py
dictionary_with_bert.py  preprocess.py	transformer_with_pretrained_bert.py
gec_data_split.py	 __pycache__	translation_with_bert.py


# Stage 1: Decoder training

In [5]:
SRC='input'
TRG='target'
BERT_MODEL='bert-base-multilingual-cased'
UPDATES_PER_EPOCH=9000
DISP_FREQ=int(UPDATES_PER_EPOCH / 5)
TOKENS_PER_GPU=10000
UPDATE_FREQ=4
TRAIN_EPOCHS=20
LR=0.0004
DROPOUT=0.15
WARMUP_EPOCHS=5
WARMUP=UPDATES_PER_EPOCH * WARMUP_EPOCHS

DATA="%s/data" % root_dir
CODE=src_dir
MODEL_OUT="%s/model_stage1" % root_dir
!mkdir -p $MODEL_OUT

In [8]:
!fairseq-train "/content/drive/My Drive/MasterThesis/CoLab/code_final/data" -s $SRC -t $TRG \
  --ddp-backend no_c10d \
  --user-dir "/content/drive/My Drive/MasterThesis/CoLab/code_final/src" \
  --task translation_with_bert \
  --bert-model $BERT_MODEL \
  --arch transformer_with_pretrained_bert \
  --log-format simple \
  --log-interval $DISP_FREQ \
  --max-tokens $TOKENS_PER_GPU \
  --update-freq $UPDATE_FREQ \
  --max-epoch $TRAIN_EPOCHS \
  --optimizer adam \
  --lr $LR \
  --adam-betas '(0.9, 0.99)' \
  --label-smoothing 0.1 \
  --clip-norm 5 \
  --dropout $DROPOUT \
  --min-lr '1e-09' \
  --lr-scheduler inverse_sqrt \
  --weight-decay 0.0001 \
  --criterion label_smoothed_cross_entropy \
  --warmup-updates $WARMUP \
  --warmup-init-lr '1e-07' \
  --reset-optimizer \
  --fp16 \
  --save-dir "/content/drive/My Drive/MasterThesis/CoLab/code_final/model_stage1" \
  2>&1 | tee "/content/drive/My Drive/MasterThesis/CoLab/code_final/model_stage1/training.log"


2020-07-28 00:22:20.365334: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.99)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_with_pretrained_bert', attention_dropout=0.0, bert_model='bert-base-multilingual-cased', best_checkpoint_metric='loss', bpe=None, bucket_cap_mb=25, clip_norm=5.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/content/drive/My Drive/MasterThesis/CoLab/code_final/data', dataset_impl=None, ddp_backend='no_c10d', decoder_attention_heads=12, decoder_embed_dim=768, decoder_embed_path=None, decoder_ffn_embed_dim=3072, decoder_input_dim=768, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=768, device_id=0, 